# Experiment Runner: QRC-ESN vs. Classical ESN

### **Notebook Objective**

This notebook serves as the main script to run a series of experiments comparing two reservoir computing models:
1.  **Quantum Reservoir Computer (QRC-ESN)**
2.  **Classical Echo State Network (ESN)**

### **Process**
- Defines the experiment configuration (data profiles, hyperparameter grids, and constants).
- Iterates through each defined data profile.
- For each profile, it performs a grid search over the hyperparameter space for both models, using parallel processing (`joblib`) for efficiency.
- It saves all collated results into a single `.csv` file for later analysis in a separate notebook.

In [1]:
# === IMPORTS AND SETUP ===
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import our custom modules from the 'src' directory
from src.data_generation import mackey_glass, generate_arma_data, generate_narma_data
from src.experiment import run_qrc_experiment, run_classical_experiment

import itertools
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm


# Jupyter magic command for automatic reloading of external modules
%load_ext autoreload
%autoreload 2

print("Libraries and modules loaded successfully.")

Libraries and modules loaded successfully.


In [2]:
# === EXPERIMENT CONFIGURATION ===

# General constants
SEED = 2025
TRAIN_FRACTION = 0.7
RESULTS_FILENAME = '../data/results_comparative.csv' # Results will be saved in the data folder

# Data profiles for analysis
data_profiles = [
    {'name': 'Classic_Chaos_(tau=17)', 'generator': mackey_glass, 'params': {'tau': 17}},
    {'name': 'Complex_Chaos_(tau=30)', 'generator': mackey_glass, 'params': {'tau': 30}},
    {'name': 'Hyperchaotic_(tau=100)', 'generator': mackey_glass, 'params': {'tau': 100}},
    {'name': 'ARMA_1_2_stochastic', 'generator': generate_arma_data, 'params': {}},
    {'name': 'NARMA10_Chaotic', 'generator': generate_narma_data, 'params': {'order': 10}},
    {'name': 'NARMA5_Chaotic', 'generator': generate_narma_data, 'params': {'order': 5}}
]

# Hyperparameter grid for the QRC-ESN model
param_grid_qrc = {
    'leakage_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
    'lambda_reg': [1e-8],
    'window_size': [4, 6, 8, 10],
    'n_layers': [2, 3, 4],
    'lag': [0]
}

# Hyperparameter grid for the Classical ESN model
param_grid_classical = {
    'reservoir_size': [50, 100, 150],
    'spectral_radius': [0.9, 1.1, 1.25],
    'sparsity': [0.1, 0.2],
    'leakage_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
    'lambda_reg': [1e-8]
}

print(f"Configuration ready. Results will be saved to: {RESULTS_FILENAME}")

Configuration ready. Results will be saved to: ../data/results_comparative.csv


In [3]:
# === MAIN EXECUTION LOOP ===

all_results = []

# Iterate over each defined data profile
for profile in data_profiles:
    print(f"\n{'='*20}\nSTARTING PROFILE: {profile['name']}\n{'='*20}")
    
    # Generate the time series for the current profile
    time_series = profile['generator'](**profile['params'])
    
    # --- QRC Grid Search ---
    param_combinations_qrc = list(itertools.product(*param_grid_qrc.values()))
    
    qrc_results = Parallel(n_jobs=-1)(
        delayed(run_qrc_experiment)(params, profile, time_series, TRAIN_FRACTION, SEED)
        for params in tqdm(param_combinations_qrc, desc=f"QRC Grid Search ({profile['name']})")
    )
    all_results.extend(filter(None, qrc_results))

    # --- Classical ESN Grid Search ---
    param_combinations_classical = list(itertools.product(*param_grid_classical.values()))
    
    classical_results = Parallel(n_jobs=-1)(
        delayed(run_classical_experiment)(params, profile, time_series, TRAIN_FRACTION, SEED)
        for params in tqdm(param_combinations_classical, desc=f"Classical ESN Search ({profile['name']})")
    )
    all_results.extend(filter(None, classical_results))

print("\n--- ALL EXPERIMENTS COMPLETED ---")


STARTING PROFILE: Classic_Chaos_(tau=17)


QRC Grid Search (Classic_Chaos_(tau=17)):   0%|          | 0/60 [00:00<?, ?it/s]

Classical ESN Search (Classic_Chaos_(tau=17)):   0%|          | 0/90 [00:00<?, ?it/s]


STARTING PROFILE: Complex_Chaos_(tau=30)


QRC Grid Search (Complex_Chaos_(tau=30)):   0%|          | 0/60 [00:00<?, ?it/s]

...

Classical ESN Search (NARMA10_Chaotic):   0%|          | 0/90 [00:00<?, ?it/s]


STARTING PROFILE: NARMA5_Chaotic


QRC Grid Search (NARMA5_Chaotic):   0%|          | 0/60 [00:00<?, ?it/s]

Classical ESN Search (NARMA5_Chaotic):   0%|          | 0/90 [00:00<?, ?it/s]


--- ALL EXPERIMENTS COMPLETED ---


In [4]:
# === SAVE RESULTS ===

# Convert the list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame(all_results)

# Save the DataFrame to a CSV file
results_df.to_csv(RESULTS_FILENAME, index=False)

print(f"Successfully saved {len(results_df)} results to {RESULTS_FILENAME}")
results_df.head()

Successfully saved 900 results to ../data/results_comparative.csv


,model_type,data_profile,mse,leakage_rate,lambda_reg,window_size,n_layers,lag,seed,reservoir_size,spectral_radius,sparsity
0,QRC,Classic_Chaos_(tau=17),0.001060,0.1,1.000000e-08,4,2.0,0.0,2025,NaN,NaN,NaN
1,QRC,Classic_Chaos_(tau=17),0.000826,0.1,1.000000e-08,4,3.0,0.0,2025,NaN,NaN,NaN
2,QRC,Classic_Chaos_(tau=17),0.000970,0.1,1.000000e-08,4,4.0,0.0,2025,NaN,NaN,NaN
3,QRC,Classic_Chaos_(tau=17),0.000470,0.1,1.000000e-08,6,2.0,0.0,2025,NaN,NaN,NaN
4,QRC,Classic_Chaos_(tau=17),0.000578,0.1,1.000000e-08,6,3.0,0.0,2025,NaN,NaN,NaN
